In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

import os
import sys
import warnings
warnings.simplefilter('ignore')

from IPython.display import display

# Add modules paths to system path
module_paths = ['.', '/usr/local/lib/python3.8/site-packages']
for module_path in module_paths:
    if os.path.abspath(os.path.join(module_path)) not in sys.path:
        sys.path.append(module_path)
    
import pickle
import numpy as np
import pandas as pd
import builtins

import matplotlib.pyplot as plt

from ipywidgets import interact, fixed
import ipywidgets as widgets

from struct_opt_dms.utils import save, load
from struct_opt_dms.dataset import load_dataframe
from struct_opt_dms.interactive_analysis import initialize, prep_power, select_frequency_range

current_input_id = -1

input_filterbanks_repository = '../data/filterbanks/R3/'
# state_variable_name = 'df_arts_R3_gaussian_errors'
state_variable_name = 'df_arts_R3'
reload_state = True

# Prepare bursts metadata
(detection parameters, repository name, local and arts paths) 

In [ ]:
df_R3 = load_dataframe(input_filterbanks_repository, state_variable_name, reload=reload_state)
# df_missing = df_R3.loc[df_R3['struct_opt_dm'].isnull()]

In [ ]:
df_R3

# Load input data

In [ ]:
# Main
verbose = True

# Go from one file to the other in the list in ascending (True) or decending (False) order
# `None` will reload the same file.
incr = True

current_input_id = -1

if incr:
    current_input_id += 1
elif incr is not None:
    current_input_id -= 1
    
current_input_id = df_R3.loc[df_R3['paper_name'] == 'A17'].index[0]
    
print ('Current burst (index=%d):' % current_input_id)
print (df_R3.iloc[current_input_id])
print ()

if df_R3.iloc[current_input_id]['file_location'] == '':
    if incr:
        current_input_id += 1
    elif incr is not None:
        current_input_id -= 1
            
spectra, dm_trials, filename = initialize(df_R3.iloc[current_input_id]['file_location'], 
                                          df_R3.iloc[current_input_id]['detection_dm'],
#                                           348.8,
                                          df_R3.iloc[current_input_id]['detection_downsampling'],
                                          around_peak=False,
                                          verbose=verbose)
detection_mjd = df_R3.iloc[current_input_id]['detection_mjd']
power_vs_dm, d_power_vs_dm = prep_power(spectra, dm_trials, verbose=verbose)


# Interactive visualisation

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(10, 7))
# plt.rcParams["font.family"] = "Times New Roman"
# plt.rcParams["font.size"] = "12"
gs = fig.add_gridspec(10, 6)

def load_value(param, df, default=0):
    if pd.isnull(df.loc[df['detection_mjd'] == detection_mjd, param].values[0]):
        return default
    else:
        return df.loc[df['detection_mjd'] == detection_mjd, param]

interact(
    select_frequency_range, 
    spectra = fixed(spectra), 
    dm_trials = fixed(dm_trials),
    power_vs_dm = fixed(power_vs_dm),
    d_power_vs_dm  = fixed(d_power_vs_dm),
    fig=fixed(fig),
    gs=fixed(gs),
    descriptor = fixed(df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'paper_name'].values[0]),
    fitting_method = widgets.Dropdown(options=['Gaussian', 'dm_phase'], value='Gaussian'),
    fluct_id_low = widgets.IntSlider(min = 0, 
                                     max = power_vs_dm.shape[0], 
                                     step = 1, 
                                     value = load_value('fluct_id_low', df_R3, 0), 
                                     continuous_update=False), 
    fluct_id_high = widgets.IntSlider(min = 0, 
                                      max = power_vs_dm.shape[0], 
                                      step = 1, 
                                      value = load_value('fluct_id_high', df_R3, power_vs_dm.shape[0]), 
#                                       value = 110,
                                      continuous_update=False), 
    freq_id_low = widgets.IntSlider(min = 0, 
                                    max = spectra.data.shape[0], 
                                    step = 1, 
                                    value = load_value('freq_id_low', df_R3, 0), 
                                    continuous_update=False),
    freq_id_high = widgets.IntSlider(min = 1, 
                                     max = spectra.data.shape[0], 
                                     step = 1, 
                                     value = load_value('freq_id_high', df_R3, spectra.data.shape[0]), 
                                     continuous_update=False),
    t0 = widgets.IntSlider(min = 0, 
                           max = spectra.data.shape[1], 
                           step = 1, 
                           value = load_value('t0', df_R3, 0), 
#                            value = 550,
                           continuous_update=False), 
    t1 = widgets.IntSlider(min = 1, 
                           max = spectra.data.shape[1], 
                           step = 1, 
                           value = load_value('t1', df_R3, spectra.data.shape[1]), 
#                            value = 671,
                           continuous_update=False),
    ds_freq = widgets.IntSlider(min=1, 
                                max=32, 
                                step=1, 
                                value=load_value('ds_freq', df_R3, 1), 
                                continuous_update=False),
    ds_time = widgets.IntSlider(min=1, 
                                max=32, 
                                step=1, 
                                value=load_value('ds_time', df_R3, 1), 
                                continuous_update=False),
    delta_dm = widgets.FloatSlider(min = -10, 
                                   max = 10, 
                                   step = 0.01, 
                                   value = load_value('delta_dm', df_R3, 0), 
                                   continuous_update=False),
    smooth = widgets.IntSlider(min=0, 
                               max=4, 
                               step=1, 
                               value=load_value('smooth', df_R3, 0), 
                               continuous_update=False),
    
)

# Save button
button = widgets.Button(description="Save figure")
display(button)

button_raw = widgets.Button(description="Save raw figure")
display(button_raw)

button_clear = widgets.Button(description="Clear params")
display(button_clear)

def save_to_df():
    print (detection_mjd)
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'struct_opt_dm'] = builtins.struct_opt_dm 
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'struct_opt_dm_err'] = builtins.struct_opt_dm_err
    
    # Widgets values
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'fluct_id_low'] = builtins.fluct_id_low
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'fluct_id_high'] = builtins.fluct_id_high
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'freq_id_low'] = builtins.freq_id_low
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'freq_id_high'] = builtins.freq_id_high
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 't0'] = builtins.t0
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 't1'] = builtins.t1
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'ds_freq'] = builtins.ds_freq
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'ds_time'] = builtins.ds_time
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'delta_dm'] = builtins.delta_dm
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'smooth'] = builtins.smooth
    
    save(state_variable_name, df_R3)
    df_R3.to_csv('arts_r3.csv', index=False)

def check_dir(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    
def save_figure(b):
    check_dir('images/manual_opt')
    check_dir('images/manual_opt/data')
    
    _filename = df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'paper_name'].values[0]
    
    print ("saved to images/manual_opt/%s.png" % _filename)
    fig.savefig("images/manual_opt/%s.png" % _filename, dpi=300)

    with open("images/manual_opt/data/%s_waterfall.npy" % _filename, 'wb') as f:
        np.savetxt(f, builtins.sub_waterfall)
    with open("images/manual_opt/data/%s_fluctuation.npy" % _filename, 'wb') as f:
        np.savetxt(f, builtins.power_vs_dm)
        
    save_to_df()
        
def save_figure_raw(b):
    check_dir('images/manual_opt/raw')
    check_dir('images/manual_opt/raw/data')
    
    _filename = df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'paper_name'].values[0]
    
    print ("saved to images/manual_opt/raw/%s.png" % _filename)
    fig.savefig("images/manual_opt/raw/%s.png" % _filename, dpi=300)

    with open("images/manual_opt/raw/data/%s_waterfall.npy" % _filename, 'wb') as f:
        np.savetxt(f, builtins.sub_waterfall)
    with open("images/manual_opt/raw/data/%s_fluctuation.npy" % _filename, 'wb') as f:
        np.savetxt(f, builtins.power_vs_dm)
        
    save_to_df()
    
def clear_struct_params(b):
    for param in ['struct_opt_dm', 'struct_opt_dm_err', 'fluct_id_low', 'fluct_id_high', 'freq_id_low',
       'freq_id_high', 't0', 't1', 'ds_freq', 'ds_time', 'delta_dm', 'smooth']:
        df_R3.loc[df_R3['detection_mjd'] == detection_mjd, param] = np.nan
        
    save(state_variable_name, df_R3)
    print ('Parameters cleared')

button.on_click(save_figure)
button_raw.on_click(save_figure_raw)
button_clear.on_click(clear_struct_params)
